In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
device = ("cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu")

print(f"Using {device} device")

Using cuda device


In [14]:
# Define model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



In [15]:
class ResNet(MLP):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) + x
        return logits

In [16]:
model = ResNet().to(device)
print(model)

ResNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [17]:
if __name__ == "__main__":
    exp_name = 'near90deg/'
    data = pd.read_csv('../data/pendulum_exps/'+exp_name+'traindata.csv')
    dim = 2
    print(data.head()) 

   Pendulum Angle (rad)  Angular Velocity (rad/s)  Pendulum Angle next (rad)  \
0              1.292684                  0.000000                   1.292636   
1              1.292636                 -0.009616                   1.292492   
2              1.292492                 -0.019231                   1.292252   
3              1.292252                 -0.028846                   1.291915   
4              1.291915                 -0.038460                   1.291483   

   Angular Velocity next (rad/s)  
0                      -0.009616  
1                      -0.019231  
2                      -0.028846  
3                      -0.038460  
4                      -0.048073  


In [18]:
    # Extract input features (X) and target labels (y)
    X = data[['Pendulum Angle (rad)', 'Angular Velocity (rad/s)']].values  # Input features
    y = data[['Pendulum Angle next (rad)', 'Angular Velocity next (rad/s)']].values  # Target labels
    
    print(X[0],y[0])

[1.2926844 0.       ] [ 1.29263633 -0.00961571]


In [19]:
    # Split the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Size of training = ', X_train.shape[0]);
    print('Size of testing = ', X_test.shape[0]);

Size of training =  79920
Size of testing =  19980


In [20]:
    # Convert NumPy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [21]:
    # Create PyTorch datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_dataloader:
    
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_dataloader):.4f}')
    
    print('Training finished!')

Epoch 1/10, Loss: 0.0002
Epoch 2/10, Loss: 0.0000
Epoch 3/10, Loss: 0.0000
Epoch 4/10, Loss: 0.0000
Epoch 5/10, Loss: 0.0000
Epoch 6/10, Loss: 0.0000
Epoch 7/10, Loss: 0.0000
Epoch 8/10, Loss: 0.0000
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000
Training finished!


In [24]:
    # Evaluation
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_dataloader:
    
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs.squeeze(), targets).item()
    test_loss /= len(test_dataloader)
    print(f'Test Loss: {test_loss:.4f}')

Test Loss: 0.0000


In [25]:
    torch.save(model.state_dict(), '../models/'+'pendulum_trained_resnet_near90deg.pth')
    print("Saved PyTorch Model State to pendulum_trained.pth")

Saved PyTorch Model State to pendulum_trained.pth
